In [7]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ipywidgets

  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("No OpenAI API key found. Please check your .env file.")

# Set the API key in environment and initialize client
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)


In [ ]:
# --- Step 1: Interactive wound-care intake (checkboxes + variable binding) ---

from ipywidgets import widgets, VBox, Button, Layout, Output, Label
from IPython.display import display, clear_output

print("### Wound Assessment Prototype ###\n")

# --- Supplies (checkboxes) ---
supply_options = ["Gauze", "Xeroform", "Tape", "Saline", "Antibiotics", "Bandages", "Gloves", "Other"]
supply_checks = [widgets.Checkbox(value=False, description=opt) for opt in supply_options]
supplies_box = VBox(supply_checks)

def get_supplies():
    return [c.description for c in supply_checks if c.value]

# --- Setting (dropdown) ---
setting_widget = widgets.Dropdown(
    options=["Harm reduction clinic", "Outpatient clinic", "Home", "Other"],
    description="Setting:",
    style={'description_width': 'initial'},
)

# ✅ Updated Expertise Level Dropdown
expertise_widget = widgets.Dropdown(
    options=["Healthcare professional with wound care experience", "Healthcare professional without wound care experience", "Non-healthcare professional"],
    description="Expertise level with these wounds:",
    style={'description_width': 'initial'},
)

# --- Willingness (radio buttons) ---
willingness_widget = widgets.RadioButtons(
    options=["Yes", "No"],
    description="Willing to go to hospital?",
    style={'description_width': 'initial'},
)

# --- Frequency (dropdown) ---
frequency_widget = widgets.Dropdown(
    options=["Daily", "Weekly", "Other"],
    description="Clinic visit frequency:",
    style={'description_width': 'initial'},
)

# --- Infection status (radio buttons) ---
infected_widget = widgets.RadioButtons(
    options=["Yes", "No", "Not sure"],
    description="Wound infected?",
    style={'description_width': 'initial'},
)

# --- Moisture (radio buttons) ---
moisture_widget = widgets.RadioButtons(
    options=["Dry", "Wet"],
    description="Wound moisture:",
    style={'description_width': 'initial'},
)

# --- Image path (text input) ---
image_widget = widgets.Text(
    value='',
    placeholder='/Users/you/Downloads/wound.jpg',
    description='Image path:',
    style={'description_width': 'initial'},
)

# --- Output + submit logic ---
out = Output()

def on_submit(_):
    with out:
        clear_output()
        inputs = {
            "supplies": get_supplies(),
            "setting": setting_widget.value,
            "expertise": expertise_widget.value,
            "willingness": willingness_widget.value,
            "frequency": frequency_widget.value,
            "infected": infected_widget.value,
            "moisture": moisture_widget.value,
            "image_path": image_widget.value
        }
        globals().update(inputs)
        print("✅ Inputs collected successfully:\n")
        for k, v in inputs.items():
            print(f"{k}: {v}")
        print("\nYou can now run the next analysis cell directly.")

submit_button = Button(
    description="Submit",
    button_style="success",
    layout=Layout(width="200px")
)
submit_button.on_click(on_submit)

# --- Layout ---
ui = VBox([
    Label("Supplies (check all that apply):"),
    supplies_box,
    setting_widget,
    expertise_widget,  # Replaced comfort slider
    willingness_widget,
    frequency_widget,
    infected_widget,
    moisture_widget,
    image_widget,
    submit_button,
    out
])

display(ui)


### Wound Assessment Prototype ###



In [ ]:
# Step 2 — Convert image to base64
import base64
from IPython.display import Markdown, display

try:
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    image_b64 = base64.b64encode(image_bytes).decode("utf-8")
except Exception as e:
    print(f"⚠️ Error loading image: {e}")
    image_b64 = None

# Step 3 — Construct dynamic educational prompt
prompt = f"""
You are an educational AI assistant helping to identify visual features in wound images
for research and model development.

Your goal is to generate a step-by-step treatment plan **based on both the text context and the visible landmarks in the image.**


User-provided context:
- Supplies available: {supplies}
- Setting: {setting}
- Expertise-level: {expertise}
- Willing to visit hospital: {willingness}
- Frequency of clinic visits: {frequency}
- Wound infection status: {infected}
- Wound moisture: {moisture}

### Instructions ###
1. Carefully examine **visual landmarkers** in the wound image — e.g., color changes, necrotic tissue, swelling, drainage, redness, or exposed structures.
2. Incorporate those landmarks explicitly into the treatment plan (e.g., "Clean around the dark necrotic edge" or "Protect the red granulating area with Xeroform").
3. Use only supplies the user has available. Do not use supplies if its excessive for the severity of the wound. If warranted include a statement at the end that details what other supplies would be needed for optium care.
4. Do not use em dashes (—), en dashes (–), or hyphens (-) for separating phrases; instead use commas or semicolons.
5. Carefully consider the expertise-level when choosing the language for the instructions
6. Keep your output as a **numbered list** (1., 2., 3., etc.) with concise, actionable wound-care steps.
"""

# Step 4 — Send to GPT-4 Vision
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
        ]
    }
]

# Attach image if present
if image_b64:
    messages[0]["content"].append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{image_b64}"
        }
    })

try:
    # Use GPT-4 Vision API
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
    )
    
    # Extract and display the response
    text = response.choices[0].message.content
    print("\n### Wound Description (Educational Use) ###\n")
    display(Markdown(text))
    
except Exception as e:
    print(f"⚠️ Error calling OpenAI API: {e}")


### Wound Description (Educational Use) ###



1. Put on gloves before touching the wound or surrounding skin.
2. Irrigate the wound thoroughly with saline, focusing on rinsing away debris and any exudate from the exposed red areas between the black necrotic patches and along the wound margins.
3. Gently pat the dry peripheral skin and intact areas with sterile gauze, avoiding aggressive friction on the fragile edges and the exposed tissue.
4. Do not attempt to debride the black necrotic tissue yourself in this setting; avoid disturbing the thick eschar, but ensure the surrounding red moist areas are kept as clean as possible.
5. Apply a layer of Xeroform gauze, particularly covering the exposed red and yellow areas as well as the edges around necrotic tissue, to preserve moisture and reduce desiccation.
6. Secure the Xeroform in place with dry gauze and then tape, ensuring you do not tape directly on damaged or fragile skin.
7. If systemic antibiotics have been prescribed for this infected wound, initiate therapy as directed; escalate care if there are signs of spreading cellulitis, systemic symptoms, or failure to improve.
8. Advise the patient to return for daily dressing changes and further wound assessment; monitor closely for increased redness, warmth, swelling, or discharge.
9. Recommend urgent referral or transfer to hospital for surgical evaluation given the extent of necrosis, infection, and involvement of a large area, which may require advanced debridement and intravenous antibiotics.

**For optimal care, advanced wound dressings for moisture balance, topical antimicrobial dressings, and access to surgical debridement would be indicated.**